# 第 8 章：数值稳定性与复杂度（Numerical Stability & Complexity）

本章关注：

- 浮点数的误差模型
- 稳定 softmax / LayerNorm
- 算法复杂度（时间/空间）与推理性能

---

## 8.1 浮点数与舍入误差（Floating-Point Error Model）
### 8.1.1 浮点模型 recap

一个典型的浮点运算可以抽象为：

$$
\text{fl}(x \circ y) = (x \circ y)(1 + \delta), \quad |\delta| \le \epsilon
$$


- $\circ$：基本运算（+,-,×,÷）
- $\epsilon$：机器精度（machine epsilon）
- $\delta$：舍入误差

连续多次运算后，误差会积累，有时还会被放大。（32位单精度浮点数中，通常有1位符号位，8位指数位，以及23位尾数）
常见精度对比：
| 格式   | mantissa bits（尾数位数） | ε（误差量级） | 是否常用在推理                     |
| ---- | ------------- | ------- | --------------------------- |
| FP32 | 23            | 1e-7    | ✓ accumulator / LN / matmul |
| FP16 | 10            | 1e-3    | ✓ 推理主力                      |
| BF16 | 7             | 1e-2    | ✓ 大模型更稳、训练更常用               |
| FP8  | 4~5           | 1e-1    | ✓ 新硬件（Hopper、Blackwell）推理常用 |

### 8.1.2 工程中误差累积的真实案例
案例 1：长序列注意力中，softmax 溢出导致全 NaN

场景：T = 8192, d = 128, FP16

如果使用 naive softmax，exp(80) 这类数会直接 ∞

后续 A V 中全是 ∞，导致最终输出 NaN

FlashAttention 论文中明确强调：

> 必须 block-wise 维护最大值，增量式 stable softmax，否则 FP16 必炸。
---

案例 2：KV cache 累加导致误差偏移（大模型）

FP16 KV cache 在 decoder 中反复参与 QK^T
时间步多时，累加方差失真：

- 注意力分布慢慢偏向少量 token

- 导致“长对话丢记忆”现象
解决：使用 FP16/INT8 权重，但 KV cache 保留 FP32 accumulator。 OpenAI、Anthropic 都是这样做。
---

**【工程意义】**  

- 在 从 FP32 → FP16 → FP8，ε 从 1e-7 → 1e-3 → 1e-1，误差放大 10,000 倍。
- 在长序列、深网络中，累积误差不容忽视
- 需要设计数值稳定的实现方式（见 softmax 示例）

### 8.1.3 Python  FP16 与 FP32 的误差差异


In [17]:
import torch

# 构造一个累计求和的例子
x = torch.ones(10000) * 1e-3
print("element value", x[0].item())

s_fp32 = x.float().sum()
s_fp16 = x.half().sum()

print("FP32:", s_fp32.item())
print("FP16:", s_fp16.item())

f = torch.tensor(1e-3)  # default to float32
print("printed:", f)
print("as float:", float(f))
print("high precision:", format(float(f), ".10f"))

h = torch.tensor(1e-3).half()
print("printed:", h)              
print("as float:", float(h))
print("high precision:", format(float(h), ".10f"))


element value 0.0010000000474974513
FP32: 10.000000953674316
FP16: 10.0078125
printed: tensor(0.0010)
as float: 0.0010000000474974513
high precision: 0.0010000000
printed: tensor(0.0010, dtype=torch.float16)
as float: 0.0010004043579101562
high precision: 0.0010004044


✅ 为什么 FP16 会偏大？
1. FP16 的 mantissa 只有 10 bits（≈3 位十进制精度）
这意味着 FP16 无法准确表示 1e-3。

- 实际 FP16 存储的值并不是 0.0010000，
- 而是最接近它的 representable number。实际存的是 0.0010004044，即比 0.001 稍大
- FP16 累加过程中每一步还有舍入误差（round-to-nearest-even）
    - 每一步都可能往“最近的偶数尾数”舍入
    - 如果中间值恰好在两边之间偏右，就会被往上舍入

大量重复后，就是现在的结果：
FP16: 10.0078125  （比 FP32 稍大）

【工程意义】
非常关键的一点结论：

即便是看起来“很小”的数，在低精度下也经常是被系统性“放大”或者“压缩”后的近似值。

配合长序列 / 大 batch / 大维度的累加，就会导致：

- LayerNorm / RMSNorm 求和有系统性 bias

- attention score 累积偏移

- loss / logit 统计量偏掉，debug 很难肉眼看出

工业做法：

- 输入可以是 FP16 / BF16 / FP8

- 所有重要的累加（sum / mean / variance / matmul 的 accumulator）都用 FP32

- 做完再 cast 回低精度存和算


✅ 为什么 FP16 不能精确表示 0.001？
根本原因：0.001 的二进制展开是无限不循环小数，而 FP16 只能存有限 10 bit 小数

1. FP16 浮点格式回顾（IEEE 754 half precision）

| 部分                  | bits        |
| ------------------- | ----------- |
| sign                | 1           |
| exponent            | 5           |
| mantissa (fraction) | **10 bits**  尾数精度 11 bits （有10位被显式存储） |

浮点数表示的是：$\text{value} = (-1)^s \cdot (1 + m) \cdot 2^{e - 15}$

其中：

- 𝑚有 10 bits
- FP16 的有效十进制精度约为 3 到 4 位有效数字
- 指数位 采用偏移二进制表示法，偏移量（Bias）为 15。实际指数值等于存储值减去 15
- 可表示的范围约为 \(2^{-14}\) 到 \(2^{15}\)

2. 0.001 转成二进制是什么？
```block
0.001 decimal = 0.0000000000000000000001010001111010111000010100011110... (binary)
```
二进制能被精确表示的十进制分数满足： 分母必须是 2 的幂。
例如：

- 0.5 = 1/2 → 可以表示

- 0.25 = 1/4 → 可以

- 0.75 = 3/4 → 可以

- 0.1 = 1/10 → ❌ 不能

- 0.001 = 1/1000 → 不能（1000=2³×125，其中125不是2的幂）

3. 因为 FP16 只能容纳 10 bits mantissa → 只能“非常粗糙地”逼近它
PyTorch 实际存的值：0.0010004044
$$0.0010004044 \approx 1/999.6$$
它比真正的 0.001 稍微大一点。

4. 计算最接近的 FP16 表示：数学验证
推导FP16 在这一段的“间隔”（ULP:Unit of Least Precision）最低有效位单元
- ULP 是数值分析中的一个核心概念，表示一个浮点数与其相邻的下一个可精确表示的浮点数之间的最小距离或步长。是改变 mantissa 最低 bit（LSB）时，浮点值的绝对差。
- ULP 的大小并非固定不变，而是取决于浮点数本身的当前量级（即其指数值）。离零越近，FP16 数字之间的间距越小；离零越远，间距越大。
- 对于一个规格化normalized的 FP16 数 $x$, 其 ULP 可以用以下公式表示：
$$\text{ULP}(x)=2^{\text{e}-\text{mantissa}+1}$$
将 FP16 的具体参数代入：
- 尾数位数（含隐式位）为 \(p=11\) 比特。
- 实际指数值为 \(E-15\)

$$ \text{ULP}(x)=2^{(e-15)-11+1}=2^{(e-25)}$$
其中 \(e\) 是存储在指数域的 5 比特整数值。

理解成$\text{ULP}(x)=(\text{尾数的最小增量})\times (\text{由指数决定的比例因子})$

把 0.001 用二进制科学计数法写出来：
$0.001 \approx 0.0010004044 = 1.0244 \times 2^{-10}$
也就是说：
$$e - bias = -10$$
$$e = -10 + 15 = 5$$


带入公式计算 FP16 在此处的 ULP
$$ULP = 2^{(e - 25)}$$
$$ULP = 2^{(5 - 25)} = 2^{-20}$$
这正好是：
$0.00000095367431640625$

而你看到的误差：
$$0.0010004044 - 0.001 = 4.044 \times 10^{-7}$$
正好是半个 step 多一点。

这说明：

0.001 的 FP16 表示就是它附近最接近的 FP16 decimal point

In [18]:
import torch
import numpy as np

x = torch.tensor(1e-3).half()

# 用 numpy 的 nextafter 找 half 精度对应 float 的邻近值
down16 = np.nextafter(x, np.float16(0), dtype=np.float16)
up16   = np.nextafter(x, np.float16(1), dtype=np.float16)

# validate ULP around 0.001 are very close to 2^-20~=9.5367×10⁻⁷
print("down:", float(down16))
print("x:", float(x))
print("up:", float(up16))
print("ULP down:", float(x) - float(down16))
print("ULP up:", float(up16) - float(x))

down: 0.00099945068359375
x: 0.0010004043579101562
up: 0.0010013580322265625
ULP down: 9.5367431640625e-07
ULP up: 9.5367431640625e-07


/var/folders/n3/5t978bxj59ddk47g5b0wqg4h0000gn/T/ipykernel_23870/1481770706.py:7: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  down16 = np.nextafter(x, np.float16(0), dtype=np.float16)
/var/folders/n3/5t978bxj59ddk47g5b0wqg4h0000gn/T/ipykernel_23870/1481770706.py:8: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  up16   = np.nextafter(x, np.float16(1), dtype=np.float16)



## 8.2 稳定 softmax
### 8.2.1 数值问题来源
朴素写法：

$$
\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}
$$

如果最大值很大，例如 80：

FP16 的 exp(80) 立即溢出为 inf

分母也是 inf，产生 NaN、

- 当某个 $x_k$ 很大时，$e^{x_k}$ 溢出 
- 其它项相对变成 0，导致数值问题

### 8.2.1 稳定写法
稳定写法：

$$
\text{softmax}(x_i) = \frac{e^{x_i - m}}{\sum_j e^{x_j - m}}, \quad m = \max_j x_j
$$



推导：

$$
\frac{e^{x_i}}{\sum_j e^{x_j}}
= \frac{e^{x_i - m} e^m}{\sum_j e^{x_j - m} e^m}
= \frac{e^{x_i - m}}{\sum_j e^{x_j - m}}
$$


- 分子分母同时乘以 $e^{-m}$，数值上避免了溢出/下溢

**【工程视角】**  

- 在 FP16/FP8 或混合精度中，这种稳定化处理尤其重要
- FlashAttention 等算法在实现时非常强调：
  - 按 block 计算 max
  - 增量式维护稳定 softmax

---
### 8.2.3 PyTorch naive vs stable 对比

In [19]:
import torch
import torch.nn.functional as F

x = torch.tensor([1000.0, 999.0, 998.0])

print("Naive softmax:")
try:
    naive = torch.exp(x) / torch.exp(x).sum()
    print(naive)
except Exception as e:
    print("Error:", e)

print("\nStable softmax:")
print(F.softmax(x, dim=-1))

Naive softmax:
tensor([nan, nan, nan])

Stable softmax:
tensor([0.6652, 0.2447, 0.0900])


### 8.2.4 工程级稳定 softmax（FlashAttention 风格）
核心技巧：

- block 级最大值维护

- 增量式 log-sum-exp 更新
伪代码
```python
for block in blocks:
    m_new = max(m, max(block))
    s = s * exp(m - m_new) + sum(exp(block - m_new))
    m = m_new
```

**flash-attention 最重要不是减少 FLOPs，而是： 保持 softmax 全程数值稳定（FP16/FP8**



## 8.3 LayerNorm / RMSNorm 的数值考虑
### 8.3.1 LayerNorm 的 FP16 问题
LayerNorm：

$$
\mu = \frac{1}{d}\sum_{i=1}^d x_i, \quad
\sigma^2 = \frac{1}{d}\sum_{i=1}^d (x_i - \mu)^2 \\
\hat{x}_i = \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}}
$$


数值稳定性挑战：

- FP16 累加会严重损失精度

- $(x_i - \mu)^2$ 在 FP16 下可能下溢成 0

- $\sigma^2 + \epsilon$ 可能不够大，导致除法不稳定

### 8.3.2 工业界默认解决方案
方案 A：FP32 作为 accumulator，FP16 输入输出

流程：

- cast x → FP32

- 求均值与方差

- 再 cast 回 FP16

PyTorch、xFormers、TensorRT 全是这样。

---
方案 B：RMSNorm 更稳定（Transformer 默认）

$$\text{RMSNorm}(x) = \frac{x}{\sqrt{\frac{1}{d}\sum x_i^2 + \epsilon}}$$
优势：

- 不计算 mean → 少一个高方差操作

- FP16 中更稳定
所以 LLaMA 全部用 RMSNorm。
---

RMSNorm  和 LayerNorm 区别？
- LayerNorm：先减均值再除以标准差，公式 (x-mean)/sqrt(var+eps)，有可学习的 γ/β。对均值与幅度都做标准化。
- RMSNorm：只用均方根归一，不减均值，公式 x / rms(x)，通常也有可学习的缩放（有些实现无偏置）。只缩放幅度，不中心化。

实际影响：

- RMSNorm 计算更简单、略快，数值上更稳定一些（少一次减均值）；在部分大模型（如 LLaMA 系列）表现不错。
- LayerNorm 更常见，输出零均值、单位方差，对均值漂移也做了归一，一些任务上可能更稳。

在大多数 Transformer 任务中，两者精度差异不大，需要实测；若追求速度/简化，RMSNorm 是轻量替代。

---
🔥 为什么 FP16 中 $(x_i - \mu)^2$会下溢成 0？

1. FP16 的最小规格化数非常大（≈ 6e-8），square 操作会瞬间掉到 FP16 的非规格化区甚至直接变成 0。

计算过程：

- 最小正的 normal 数（最小正规化数）
   - 符号位：0
   - 指数位：00001（实际指数 = 1 - 15 = -14）
   - 尾数位：0000000001（隐含前导1 → 实际尾数 1.0000000001₂）
   - 值 = 2⁻¹⁴ × 1.0000000001₂ ≈ 2⁻¹⁴ × 1.000000000 = 6.103515625e-5

- 最小正的 subnormal 数（最小非正规化数 / 最小正数）
   - 符号位：0
   - 指数位：00000（固定解释为指数 -14）
   - 尾数位：0000000001（无隐含前导1 → 实际尾数 0.0000000001₂ = 2⁻¹⁰）
   - 值 = 2⁻¹⁴ × 2⁻¹⁰ = 2⁻²⁴ ≈ 5.9604644775390625e-8

也就是说小于6e-8的数会下溢到0

---

2. LayerNorm / RMSNorm 计算本质含大量极小数

在 LN 中：$x_i - \mu$
如果某个$x_i$很接近均值$\mu$，比如：
- $1e-3 - 1.000001e-3 = -1e-9$
- $0.035412 - 0.035411 = 1e-6$
- $0.004 - 0.00400003 = -3e-8$
这些在 FP32 完全没问题。

但在 FP16中任何$|x_i - \mu| < 6e-8$的情况，直接变成 0。

---

3. 接下来 LN 要做平方：
$$(x_i - \mu)^2$$
如果$|x_i - \mu|$本来是：
- $1e-6$
- $1e-7$
- $1e-8$
平方后分别变成：
- $1e-12$
- $1e-14$
- $1e-16$
这些数全部：远远 小于 FP16 的最小正数（subnormal） 5.96e-8 → 直接下溢成 /或被量化为0。

这意味着：
$$\sigma^2 = \frac{1}{d}\sum (x_i - \mu)^2 = 0$$
LN 输出变成：

$$\frac{x_i - \mu}{\sqrt{0 + \epsilon}} \approx 0$$
最终 LN 整层会输出一堆几乎相同的值 → 信息被抹掉 → 模型稳定性下降、推理效果变差。

In [20]:
# pytorch to reproduce FP16 variance disappearance issue
import torch

x = torch.tensor([1.0000, 1.0001, 0.9999, 1.00005], dtype=torch.float16)
mu = x.mean()
sq = (x - mu) ** 2

print("FP16 x:", x)
print("FP16 mean:", mu)
print("FP16 variance terms:", sq)



FP16 x: tensor([1., 1., 1., 1.], dtype=torch.float16)
FP16 mean: tensor(1., dtype=torch.float16)
FP16 variance terms: tensor([0., 0., 0., 0.], dtype=torch.float16)


### 8.3.4 工程结论

- FP16/FP8 累加必须使用 FP32 accumulator

     - LayerNorm 累加
 
     - RMSNorm 累加

     - Attention 中的 QK、AV 累加

     - Softmax 的指数和求和

     - KV cache decaying 累加

     - matmul 内部 partial sum

流程是：

- cast FP16 → FP32

- FP32 做 mean

- FP32 做平方

- FP32 做 variance

- 最后 cast 回 FP16

## 8.4 算法复杂度（Complexity）
### 8.4.1 为什么复杂度对推理至关重要'
推理瓶颈来自：
- 时间复杂度：FLOPs
- 空间复杂度：activation + KV Cache
- 带宽：从 DRAM → NPU
---
### 8.4.2 自注意力的复杂度
传统 attention：
$$O(T^2 d)$$
如果 T=4096, d=128:
- QKᵀ FLOPs ~ 4096² × 128 ≈ 2.1e9（巨大！）
- V 加权同级别
---
8.4.3 降低复杂度的典型方向
1. FlashAttention（同阶但常数更小）
   - 保留 $O(T^2)$

   - 减少 activation + memory bandwidth

   - 提升 2~4x
2. 细粒度 KV cache compression
   - top-k 记忆

   - 局部注意力

   - memory budget <-> latency tradeoff

3. 低秩近似（SVD/low-rank attention）
将 $$QK^\top ≈ U V^\top$$
复杂度从
$O(T^2 d)$ → $O(T r)$，其中 r ≪ d

4. K-NN Attention / Linear Attention
利用核技巧：
$$\text{softmax}(QK^T)V = Q (K^T V)$$
复杂度变成 $O(T d)$。


## 8.5 本章小结

1. 浮点误差模型

理解：

- FP16/FP8 会放大每个 kernel 的数值问题

- 所以部署必须严格遵循“稳定写法”

2. 稳定 softmax

必要性：

- 长序列 attention 中是第一大炸点

- FlashAttention 的核心不是加速，而是稳定

3. LayerNorm / RMSNorm

工程常用技巧：

- accumulators = FP32

- RMSNorm 比 LN 更稳

4. 算法复杂度

你要能：

- 估算 FLOPs

- 判断瓶颈（matmul / softmax / KV cache）

- 选择合适的优化（fusion / low-rank / local attention）


## 8.6 FP16（IEEE 754 binary16）关键数值一览
| 项目               | 二进制表示            | 十进制精确值                  | 常用科学计数法          |
|--------------------|-----------------------|-------------------------------|--------------------------|
| 最小正 normal 数   | 0 00001 0000000001   | 6.103515625 × 10⁻⁵           | 6.103515625e-5          |
| 最小正 subnormal 数| 0 00000 0000000001   | 5.9604644775390625 × 10⁻⁸    | 5.9604644775390625e-8   |
| 最大 subnormal 数  | 0 00000 1111111111   | 6.09765625 × 10⁻⁵            | 6.09765625e-5           |
| 最大 normal 数     | 0 11110 1111111111   | 65504                         | 6.5504 × 10⁴            |
| 正零               | 0 00000 0000000000   | 0                             | 0                       |


常见浮点格式最小正数对比表

| 格式   | 位宽分配（符号+指数+尾数） | 最小正 normal 数               | 最小正 subnormal 数                  | 是否支持 subnormal |
|--------|----------------------------|--------------------------------|---------------------------------------|---------------------|
| FP16   | 1+5+10                     | 6.103515625 × 10⁻⁵            | 5.9604644775390625 × 10⁻⁸            | 支持                |
| BF16   | 1+8+7                      | 1.17549435 × 10⁻³⁸            | 不支持（直接下溢到 0）                | 不支持              |
| FP32   | 1+8+23                     | 1.17549435 × 10⁻³⁸            | 1.40129846 × 10⁻⁴⁵                   | 支持                |
| FP64   | 1+11+52                    | 2.2250738585072014 × 10⁻³⁰⁸   | 4.9406564584124654 × 10⁻³²⁴          | 支持                |
---
| 格式 | 最小 normal         | 最小正数 (subnormal)       | 2的幂表示   |
|------|---------------------|----------------------------|-------------|
| FP16 | 6.1035e-5           | 5.9605e-8                  | 2⁻²⁴        |
| BF16 | 1.1755e-38          | 不支持                     | -           |
| FP32 | 1.1755e-38          | 1.4013e-45                 | 2⁻¹²⁶       |
| FP64 | 2.2251e-308         | 4.9407e-324                | 2⁻¹⁰⁷⁴      |


FP16 最常背的两个数字（99%的情况只用记这两个）：
- 最小 normal   → 6.103515625e-5   （约 2⁻¹⁴）
- 最小正数      → 5.9604644775e-8   （精确 2⁻²⁴）

## 8.7 FP16/FP8 数值稳定性实验合集

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
torch.set_printoptions(sci_mode=True, precision=8)

In [22]:
# FP16 vs FP32 简单累加误差
import torch

N = 10000
x = torch.ones(N) * 1e-3  # FP32

print("element (FP32):", x[0].item())
print("element (FP16):", x[0].half().item())

s_fp32 = x.float().sum()
s_fp16 = x.half().sum()

print("FP32 sum:", s_fp32.item())
print("FP16 sum:", s_fp16.item())

# 看看单个 1e-3 在 FP32 / FP16 中的真实值
v32 = torch.tensor(1e-3, dtype=torch.float32)
v16 = torch.tensor(1e-3, dtype=torch.float16)

print("FP32(1e-3) as float:", float(v32))
print("FP16(1e-3) as float:", float(v16))
print("delta per element:", float(v16) - float(v32))

element (FP32): 0.0010000000474974513
element (FP16): 0.0010004043579101562
FP32 sum: 10.000000953674316
FP16 sum: 10.0078125
FP32(1e-3) as float: 0.0010000000474974513
FP16(1e-3) as float: 0.0010004043579101562
delta per element: 4.043104127049446e-07


In [ ]:
# 随机数累加在 FP32 vs FP16 下，误差会随机游走（drift），不是简单的 cancel out。
# （随机 ±δ 的长期漂移）
torch.manual_seed(0)

N = 100000
x = (torch.rand(N) - 0.5) * 1e-3  # 零均值小噪声

s32 = x.float().sum()
s16 = x.half().sum()

print("FP32 sum:", s32.item())
print("FP16 sum:", s16.item())
print("absolute drift:", (s16 - s32).item())

FP32 sum: 0.027622273191809654
FP16 sum: 0.02764892578125
absolute drift: 2.6652589440345764e-05


In [ ]:
# 大数 + 小数在 FP16 中，小数会完全被“吃掉”。
# 因为 FP16 的有效位只有 10 位（约 11 位精度），大数和小数相加时要对齐指数：
# 小数的尾数会被右移多位再相加。如果小数的量级比大数的 ULP（单位间距）还小，就被舍入到 0，结果保持大数不变。
# 换句话说，在某个量级下 FP16 能表示的最小步长是(2^(e-10)；当小数 < 这个步长时，加法被“吃掉”
a32 = torch.tensor(1e4, dtype=torch.float32)
b32 = torch.tensor(1e-3, dtype=torch.float32)

a16 = a32.half()
b16 = b32.half()

print("FP32 a + b:", float(a32 + b32))
print("FP16 a + b:", float(a16 + b16))

print("FP32 delta:", float((a32 + b32) - a32))
print("FP16 delta:", float((a16 + b16) - a16))

FP32 a + b: 10000.0009765625
FP16 a + b: 10000.0
FP32 delta: 0.0009765625
FP16 delta: 0.0


In [25]:
# naive softmax 在大 logits 时直接 inf / nan，稳定写法没问题。
x = torch.tensor([1000.0, 999.0, 998.0])  # 极端 logits

# naive
naive_num = torch.exp(x)
naive_den = naive_num.sum()
naive = naive_num / naive_den

print("Naive exp(x):", naive_num)
print("Naive softmax:", naive)

# stable
stable = F.softmax(x, dim=-1)
print("Stable softmax:", stable)

Naive exp(x): tensor([inf, inf, inf])
Naive softmax: tensor([nan, nan, nan])
Stable softmax: tensor([6.65240943e-01, 2.44728476e-01, 9.00305733e-02])


In [26]:
# 模拟长序列时，在 FP16 下 softmax 的精度明显比 FP32 差。
torch.manual_seed(0)

T = 4096
x = torch.randn(T) * 10  # 放大一些，模拟注意力 logits

soft32 = F.softmax(x.float(), dim=-1)
soft16 = F.softmax(x.half(), dim=-1).float()

print("L1 diff:", (soft16 - soft32).abs().sum().item())
print("Max diff:", (soft16 - soft32).abs().max().item())
print("Sum32:", soft32.sum().item(), "Sum16:", soft16.sum().item())

L1 diff: 0.007571723312139511
Max diff: 0.0038841962814331055
Sum32: 1.000000238418579 Sum16: 0.9998015761375427


In [27]:
# 构造一组数值很接近的向量，观察 FP16 做 LN 时 variance 计算不稳定。
torch.manual_seed(0)

d = 1024
base = torch.ones(d)  # 全 1
eps = torch.randn(d) * 1e-3
x = base + eps

ln = nn.LayerNorm(d)

y32 = ln(x.float())
y16 = ln(x.half()).float()

print("Mean abs diff LN(FP16 vs FP32):", (y16 - y32).abs().mean().item())
print("Max abs diff:", (y16 - y32).abs().max().item())

Mean abs diff LN(FP16 vs FP32): 0.056217942386865616
Max abs diff: 0.147597536444664


In [ ]:
# 直接手写 LN 的公式，观察 (x - μ)^2 在 FP16 下被压扁。
# 多 FP16 的 variance term 会非常接近 0。
torch.manual_seed(0)

d = 16
base = torch.ones(d) * 1.0
eps = torch.linspace(-1e-3, 1e-3, d)
x = base + eps

x16 = x.half()
mu32 = x.mean()
mu16 = x16.mean()

var_terms32 = (x - mu32) ** 2
var_terms16 = (x16 - mu16) ** 2

print("x:", x)
print("mu32:", mu32.item(), "mu16:", mu16.item())
print("var terms FP32:", var_terms32)
print("var terms FP16:", var_terms16)

x: tensor([9.99000013e-01, 9.99133348e-01, 9.99266684e-01, 9.99400020e-01, 9.99533355e-01, 9.99666691e-01,
        9.99800026e-01, 9.99933362e-01, 1.00006664e+00, 1.00020003e+00, 1.00033331e+00, 1.00046670e+00,
        1.00059998e+00, 1.00073338e+00, 1.00086665e+00, 1.00100005e+00])
mu32: 1.0 mu16: 1.0
var terms FP32: tensor([9.99974304e-07, 7.51084883e-07, 5.37752271e-07, 3.59976411e-07,
        2.17757332e-07, 1.11095005e-07, 3.99894340e-08, 4.44062209e-09,
        4.44062209e-09, 4.00132762e-08, 1.11095005e-07, 2.17812968e-07,
        3.59976411e-07, 5.37839696e-07, 7.51084883e-07, 1.00009345e-06])
var terms FP16: tensor([9.53674316e-07, 9.53674316e-07, 9.53674316e-07, 2.38418579e-07,
        2.38418579e-07, 2.38418579e-07, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        9.53674316e-07, 9.53674316e-07, 9.53674316e-07, 9.53674316e-07],
       dtype=torch.float16)


In [ ]:
# 展示 RMSNorm 不减均值，在小噪声场景下比 LN 更稳定。
# RMSNorm 的 diff 会更小。
class RMSNorm(nn.Module):
    def __init__(self, d, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(d))

    def forward(self, x):
        # x: (..., d)
        norm = x.pow(2).mean(dim=-1, keepdim=True)
        x_hat = x / torch.sqrt(norm + self.eps)
        return x_hat * self.weight

torch.manual_seed(0)
d = 1024
base = torch.ones(d) * 1.0
eps = torch.randn(d) * 1e-3
x = base + eps

ln = nn.LayerNorm(d)
rms = RMSNorm(d)

y_ln32 = ln(x.float())
y_ln16 = ln(x.half()).float()

y_rms32 = rms(x.float())
y_rms16 = rms(x.half()).float()

print("LN diff (FP16 vs FP32):", (y_ln16 - y_ln32).abs().mean().item())
print("RMS diff (FP16 vs FP32):", (y_rms16 - y_rms32).abs().mean().item())

LN diff (FP16 vs FP32): 0.056217942386865616
RMS diff (FP16 vs FP32): 0.00018921459559351206


In [30]:
# 展示矩阵乘法中，如果强制用 FP16 累加，误差会迅速扩大（PyTorch 默认已经用 FP32 累加）。
torch.manual_seed(0)

m, k, n = 256, 256, 256
A = torch.randn(m, k)
B = torch.randn(k, n)

# 参考结果（FP32 matmul）
C32 = A @ B

# 强制 FP16 计算与累加
A16 = A.half()
B16 = B.half()

# 直接 matmul 仍然可能内部用 FP32 累加，手动实现 naive matmul
C16_naive = torch.zeros(m, n, dtype=torch.float16)
for i in range(m):
    for j in range(n):
        C16_naive[i, j] = (A16[i, :] * B16[:, j]).sum()

C16_naive32 = C16_naive.float()
print("Mean abs diff:", (C16_naive32 - C32).abs().mean().item())
print("Max abs diff:", (C16_naive32 - C32).abs().max().item())

Mean abs diff: 0.005231778137385845
Max abs diff: 0.03464508056640625


In [31]:
# 用简单的线性量化模拟 FP8：（per-tensor scaling）
# x_fp8 = clamp(round(x / s), -127, 127) * s，观察 dynamic range vs 精度。
torch.manual_seed(0)

def fake_fp8_quant(x, num_bits=8, symmetric=True):
    # simple per-tensor symmetric quant
    qmax = 2**(num_bits-1) - 1  # 127
    max_val = x.abs().max()
    s = max_val / qmax if max_val > 0 else 1.0
    q = torch.round(x / s).clamp(-qmax, qmax)
    x_q = q * s
    return x_q, s

x = torch.randn(10000) * 10  # 宽动态范围
x_q, s = fake_fp8_quant(x)

print("scale:", s)
print("original std:", x.std().item(), "quant std:", x_q.std().item())
print("L2 error:", (x - x_q).pow(2).mean().sqrt().item())
print("max abs error:", (x - x_q).abs().max().item())

scale: tensor(3.41990590e-01)
original std: 10.019336700439453 quant std: 10.018792152404785
L2 error: 0.09892825782299042
max abs error: 0.1709756851196289


In [ ]:
# 展示 per-channel scaling 比 per-tensor 对某些通道（例如大 / 小量级）更友好。
# FP8 per-channel scaling（比 per-tensor 更稳定）
torch.manual_seed(0)

def fake_fp8_quant_per_channel(W, dim=0, num_bits=8):
    qmax = 2**(num_bits-1) - 1
    scales = W.abs().amax(dim=dim, keepdim=True)
    scales = torch.where(scales == 0, torch.ones_like(scales), scales)
    s = scales / qmax
    q = torch.round(W / s).clamp(-qmax, qmax)
    W_q = q * s
    return W_q, s

# 模拟一个 weight 矩阵，某些列很大，某些列很小
W = torch.randn(128, 128) * torch.linspace(0.1, 10.0, 128)

W_q_tensor, s_tensor = fake_fp8_quant(W)  # per-tensor
W_q_channel, s_channel = fake_fp8_quant_per_channel(W, dim=0)  # per-channel

err_tensor = (W - W_q_tensor).abs().mean().item()
err_channel = (W - W_q_channel).abs().mean().item()

print("Per-tensor mean abs error:", err_tensor)
print("Per-channel mean abs error:", err_channel)


Per-tensor mean abs error: 0.06652781367301941
Per-channel mean abs error: 0.028985798358917236


In [ ]:
# FP8 attention logits scaling 策略
# 模拟 attention logits 在 FP8 中：
# 不 scale 的话 logits 容易饱和；
# 加合适的 scale 可保留 shape。

torch.manual_seed(0)

def fake_fp8_quant_with_scale(x, scale):
    qmax = 127
    x_scaled = x / scale
    q = torch.round(x_scaled).clamp(-qmax, qmax)
    return q * scale

T = 128
logits = torch.randn(T) * 10  # 模拟 attention logits

# 不量化
p_ref = F.softmax(logits, dim=-1)

# 直接 FP8 量化（用 per-tensor scale）
logits_q, s = fake_fp8_quant(logits)
p_q = F.softmax(logits_q, dim=-1)

# 更 aggressive scale（减小 logits 大小）
scale_manual = 4.0
logits_scaled = fake_fp8_quant_with_scale(logits, scale_manual)
p_scaled = F.softmax(logits_scaled, dim=-1)

print("KL(ref || q):", (p_ref * (p_ref / p_q).log()).sum().item())
print("KL(ref || scaled):", (p_ref * (p_ref / p_scaled).log()).sum().item())

KL(ref || q): 1.4770594134461135e-06
KL(ref || scaled): 0.0005748498952016234


In [34]:
# 模拟 decoder 中的 KV cache 多步使用，FP16 会渐渐偏离 FP32。
torch.manual_seed(0)

T = 512
d = 64
steps = 128

K = torch.randn(T, d)
V = torch.randn(T, d)
q0 = torch.randn(d)

def step(q, K, V, dtype=torch.float32):
    q = q.to(dtype)
    K = K.to(dtype)
    V = V.to(dtype)
    logits = (K @ q) / math.sqrt(d)
    attn = F.softmax(logits, dim=-1)
    out = attn @ V
    return out

q32 = q0.clone()
q16 = q0.clone()

for i in range(steps):
    q32 = step(q32, K, V, torch.float32)
    q16 = step(q16, K, V, torch.float16).float()

print("Final L2 diff:", (q32 - q16).pow(2).sum().sqrt().item())

Final L2 diff: 0.00012237243936397135


In [ ]:
# FlashAttention-style 块 softmax 的数值优势（玩具示例）
# difference should be very very tiny
torch.manual_seed(0)

T = 1024
x = torch.randn(T) * 10

# 全局 stable softmax
def global_softmax(x):
    m = x.max()
    ex = torch.exp(x - m)
    return ex / ex.sum()

# 简化 block softmax
def block_softmax(x, block_size=128):
    m = -float('inf')
    s = 0.0
    for i in range(0, len(x), block_size):
        xb = x[i:i+block_size]
        mb = xb.max()
        # 更新全局 max
        m_new = max(m, mb)
        # 按论文里的方法修正旧的 sum
        s = s * math.exp(m - m_new) + torch.exp(xb - m_new).sum().item()
        m = m_new
    # 最终再算一遍输出
    out = []
    for i in range(0, len(x), block_size):
        xb = x[i:i+block_size]
        out.append(torch.exp(xb - m) / s)
    return torch.cat(out, dim=0)

p_global = global_softmax(x)
p_block = block_softmax(x)

print("L1 diff:", (p_global - p_block).abs().sum().item())
print("Max diff:", (p_global - p_block).abs().max().item())

L1 diff: 7.480236519086247e-08
Max diff: 5.960464477539063e-08
